In [1]:
import os 
import sys
sys.path.append('/home/kalkiek/projects/reddit-political-affiliation/')
import torch
import itertools
import random
from tqdm.notebook import tqdm
from collections import defaultdict
from sklearn.metrics import accuracy_score

#from src.data.make_dataset import build_dataset
#from src.models.word2vec.User2Subreddit import User2Subreddit

from sklearn.metrics import auc, roc_curve
#from src.data.SubredditUserDataset import SubredditUserDataset
from src.features.interactions.political_comment import PoliticalComment
import bz2
import lzma
import json
#import zstandard as zstd
from json import JSONDecodeError
from collections import Counter
import math
import pandas as pd
from matplotlib import pyplot as plt
import glob
from tqdm import tqdm
import numpy as np
from datetime import datetime
from sklearn.decomposition import PCA
#from sklearn.manifold import TSNE
from tsnecuda import TSNE
from sklearn.decomposition import TruncatedSVD
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/opt/anaconda/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['random', 'datetime']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [7]:
in_file = '/shared/0/projects/reddit-political-affiliation/data/interactions/all_comments_filtered.tsv'
df_comments=pd.read_csv(in_file,header=None, names=['comment_id','parent_id','username','subreddit','created','politics','text'],sep='\t')
df_comments.drop(df_comments[df_comments['username']=='[deleted]'].index,inplace=True)

In [19]:
# read users information
cong_dir='/shared/0/projects/reddit-political-affiliation/data/conglomerate-affiliations/'
origin_train_cong = pd.read_csv(cong_dir+'train.tsv',sep='\t')
origin_dev_cong=pd.read_csv(cong_dir+'dev.tsv',sep='\t')
origin_test_cong=pd.read_csv(cong_dir+'test.tsv',sep='\t')

# kick out community first to make life easier
train_cong=train_cong[origin_train_cong['source']!='community']
dev_cong=dev_cong[origin_dev_cong['source']!='community']
test_cong=train_cong[origin_train_cong['source']!='community']

# remove duplicate across datasets, priority: flair -> self declaration 
train_user=set(train_cong['username'])
dev_user=set(dev_cong['username'])
test_user=set(test_cong['username'])
sorted_train_cong=train_cong.sort_values(["username","source"])
distinct_train_cong=sorted_train_cong.drop_duplicates(subset="username",keep="first")
distinct_train_cong=distinct_train_cong.sample(frac=1)
sorted_test_cong=test_cong.sort_values(["username","source"])
distinct_test_cong=sorted_test_cong.drop_duplicates(subset="username",keep="first")
distinct_test_cong=distinct_test_cong.sample(frac=1)
sorted_dev_cong=dev_cong.sort_values(["username","source"])
distinct_dev_cong=sorted_dev_cong.drop_duplicates(subset="username",keep="first")
distinct_dev_cong=distinct_dev_cong.sample(frac=1)
all_cong=pd.concat([distinct_train_cong,distinct_test_cong,distinct_dev_cong])

In [20]:
# build community data in conglomerate dataset
train_cong = pd.read_csv(cong_dir+'train.tsv',sep='\t')
dev_cong=pd.read_csv(cong_dir+'dev.tsv',sep='\t')
test_cong=pd.read_csv(cong_dir+'test.tsv',sep='\t')
community_train_cong=train_cong[train_cong['source']=='community']
community_dev_cong=dev_cong[dev_cong['source']=='community']
community_test_cong=train_cong[train_cong['source']=='community']
community_cong=pd.concat([community_train_cong,community_dev_cong,community_test_cong])

/opt/anaconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [22]:
# build user sets, user2politics dictionary
flair_cong=all_cong[all_cong['source']=='flair']
gold_cong=all_cong[all_cong['source']=='gold']
flair_users=set(list(flair_cong['username']))
gold_users=set(list(gold_cong['username']))
community_users=set(list(community_cong['username']))

flair_dict=dict(zip(list(flair_cong['username']),list(flair_cong['politics'])))
gold_dict=dict(zip(list(gold_cong['username']),list(gold_cong['politics'])))
community_dict=dict(zip(list(community_cong['username']),list(community_cong['politics'])))

flair_dict.update(gold_dict)
flair_dict.update(community_dict)

In [30]:
def set_source(username):
    if username in flair_users:
        return "flair"
    if username in gold_users:
        return "gold"
    if username in community_users:
        return "community"
    return "independent"
def set_politics(username):
    return flair_dict[username]

In [50]:
# remove independent, attach politic labels
df_comments['source']=df_comments['username'].apply(set_source)
filtered_comments=df_comments[df_comments['source']!='independent']
filtered_comments['politics']=filtered_comments['username'].apply(set_politics)
filtered_comments

,comment_id,parent_id,username,subreddit,created,politics,text,source
0,eypca5l,eypby8b,ninethirtyone,news,1567296005,Republican,You responded to my comment to make a point yo...,gold
1,eypca5t,t3_cy2nlj,lostapwbm,SquaredCircle,1567296006,Republican,After the fantastic energy of the crowds for T...,community
2,eypca8l,eypbz6d,OlliesFreeOxen,news,1567296007,Republican,Run away when you lose. Typical. You have your...,community
6,eypcacv,t3_cy2yrm,DonnyGoat,NYYankees,1567296010,Republican,Whose match are they at?,community
10,eypcai6,eyo886z,AliveJesseJames,SquaredCircle,1567296013,Republican,Triple H easily could've won the title back at...,community
...,...,...,...,...,...,...,...,...
137566486,c4yco,t3_4y2h,diamond,reddit.com,1145991119,Democrat,"My initial reaction to this was outrage, like ...",community
137566487,c4yy1,t3_4yud,vagif,programming,1146012019,Democrat,"HA HA! This is a joke, right ? I got it, reall...",gold
137566493,c50rk,c502q,diamond,reddit.com,1146087497,Democrat,&gt; The shameful impeachment of Bill Clinton ...,community
137566497,c52qe,c51to,diamond,reddit.com,1146178808,Democrat,&gt; Some of the other stories make it obvious...,community


In [53]:
# select top 200 subreddits
subreddit_count=filtered_comments['subreddit'].value_counts()
subreddit_to_id=dict(zip(subreddit_count[0:2000].to_dict().keys(),range(2000)))
# keep only wanted columns
user_subreddit=filtered_comments[['username','subreddit','source','politics']].reset_index(drop=True)
user_subreddit

,username,subreddit,source,politics
0,ninethirtyone,news,gold,Republican
1,lostapwbm,SquaredCircle,community,Republican
2,OlliesFreeOxen,news,community,Republican
3,DonnyGoat,NYYankees,community,Republican
4,AliveJesseJames,SquaredCircle,community,Republican
...,...,...,...,...
80702046,diamond,reddit.com,community,Democrat
80702047,vagif,programming,gold,Democrat
80702048,diamond,reddit.com,community,Democrat
80702049,diamond,reddit.com,community,Democrat


In [54]:
def reset_subreddit(subreddit):
    if subreddit in subreddit_to_id:
        return subreddit
    return 'UNK'

In [64]:
# attach subreddit id, count frequency of users' posts in each subreddit
user_subreddit['subreddit']=user_subreddit['subreddit'].apply(reset_subreddit)
user_subreddit_matrix=pd.crosstab(user_subreddit.username,user_subreddit.subreddit)
user_subreddit_matrix

subreddit,1200isplenty,2007scape,2ALiberals,2meirl4meirl,3DS,3Dprinting,40kLore,49ers,4PanelCringe,4chan,...,xboxone,xkcd,xmen,youseeingthisshit,youtube,youtubehaiku,yugioh,zelda,zen,zombies
username,,,,,,,,,,,,,,,,,,,,,
---,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-------------------c,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
----------_----,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
----------rocketman,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
---------_---------_,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzzzebras,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zzzztopportal,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zzzzz94,0,159,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [95]:
# attach politic labels to user_subreddit_matrix
all_cong=pd.concat([flair_cong[['username','source','politics']],gold_cong[['username','source','politics']],community_cong[['username','source','politics']]])
all_cong.sort_values(by=['username','source'],inplace=True)
all_cong.drop_duplicates(subset=['username'],keep='last',inplace=True)
user_subreddit_matrix.drop(columns=['UNK'],inplace=True)
user_subreddit_matrix=pd.merge(user_subreddit_matrix,all_cong,on='username')
user_subreddit_matrix.values

array([['---', 0, 0, ..., 0, 'community', 'Democrat'],
       ['-------------------c', 0, 0, ..., 0, 'flair', 'Republican'],
       ['----------_----', 0, 0, ..., 0, 'community', 'Republican'],
       ...,
       ['zzzzz94', 0, 159, ..., 0, 'community', 'Republican'],
       ['zzzzzzzz414', 0, 0, ..., 0, 'community', 'Republican'],
       ['zzzzzzzzxzzzzzzzzzz', 0, 0, ..., 0, 'community', 'Republican']],
      dtype=object)

In [20]:
X=user_subreddit_matrix.values[:,1:-2]
Y=user_subreddit_matrix.values[:,-2:]
user_name=user_subreddit_matrix.values[:,0]

In [8]:
# save ready-to-train data
X_path='user_subreddit_matrix_X.npy'
Y_path='user_subreddit_matrix_Y.npy'
user_path='user_subreddit_matrix_username.npy'
X_reduced_path='user_subreddit_reduced_50.npy'
np.save(X_path, X)
np.save(Y_path, Y)
np.save(user_path, user_name)
# X=np.load(X_path,allow_pickle=True)
# Y=np.load(Y_path,allow_pickle=True)
# X_reduced=np.load(X_reduced_path)

In [10]:
# keep users with at least 5 posts
X_idx=np.flatnonzero(X.sum(1) >=5)
X=X[X_idx,:]
Y=Y[X_idx,:]
user_name=user_name[X_idx]

In [10]:
# pca fitting, 50 components cover about 80%, 120 -> 90%, 20 -> 70%
pca = PCA(n_components=50)
X_reduced=pca.fit_transform(X)
print(pca.explained_variance_ratio_.cumsum()[-1])
np.save(X_reduced_path,X_reduced)

0.8173469175994691


In [21]:
# run cuda tsne
dv="cuda:"+str(2)
device=torch.device(dv)
torch.cuda.set_device(int(dv[-1]))
perplexity_list=[40,60,80]
verbose_list=[2,4]
for perplexity in perplexity_list:
    for verbose in verbose_list:
        X_embedded_path='user_subreddit_embedded_20_'+str(perplexity)+'_'+str(verbose)+'.npy'
        print(X_embedded_path)
        X_embedded = TSNE(n_components=2, perplexity=perplexity, verbose=verbose).fit_transform(X_reduced)
        np.save(X_embedded_path,X_embedded)

In [ ]:
# then move to 'Rendering of t-sne Results.ipynb' for visualization